In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
DIMENSIONS = (2,2)
PROJECTION_DIMENSION = 2

projections = [
    np.array([8,3]),
    np.array([4,8])
] # we omitted the angle here, since we will define which fields are striked by the beams in the following

# same ordering as in projections
beam_affected_fields = [
    [(0,0), (1,0)],
    [(0,1), (1,1)],
    [(0,0), (0,1)],
    [(0,1), (1,1), (1,0)]
]

BEGIN = 0
ITERATE_LTR = False # do we iterate from the left to the right in a projection or else?
ENABLE_BEAM_RANKING = True # if true, then optimize beams, that go through less fields first

NUM_PROJECTIONS = len(projections)
NUM_ITERATIONS = 3

In [5]:
for x,y in zip(range(PROJECTION_DIMENSION), [2,1]):
    print(x,y)

0 2
1 1


In [6]:
def starting_image(projection):
    image = np.zeros((2,2)) + np.sum(projection)/PROJECTION_DIMENSION**2
    return image

def next_projection(current):
    return (current + 1) % NUM_PROJECTIONS if ITERATE_LTR else (current - 1) % NUM_PROJECTIONS

def get_beams_ordered(beams, ITERATE_LTR):
    beams = zip(range(PROJECTION_DIMENSION), beams) if ITERATE_LTR else zip(reversed(range(PROJECTION_DIMENSION)), reversed(beams))
    return(beams)

In [13]:
iteration = 0
current_projection = BEGIN
start_image = starting_image(projections[BEGIN])
image = np.copy(start_image)

print('Startbild:')
print(start_image)

#VARIANT A: optimize beam by beam
while iteration < NUM_ITERATIONS:
    proj_count = 0
    while proj_count < NUM_PROJECTIONS:
        viewed_beams = []
        for beam_index, beam in get_beams_ordered(projections[current_projection], ITERATE_LTR):
            strikes = beam_affected_fields[PROJECTION_DIMENSION*current_projection + beam_index]
            viewed_beams.append((len(strikes), beam, strikes))
        if ENABLE_BEAM_RANKING:
            viewed_beams = sorted(viewed_beams, key=lambda o : o[0])
        for _, beam, strikes in viewed_beams:
            #recalculate:
            cells = [image[c] for c in strikes]
            #print(cells)
            new_beam = np.sum(cells)
            #optimize:
            factor = beam/new_beam # old/new
            for c in strikes:
                #print(image[c],'*',factor, '=',np.round(image[c]*factor,2))
                image[c] = image[c]*factor
        proj_count += 1
        current_projection = next_projection(current_projection)
    print('Iteration: %s' % (iteration+1))
    print(np.round(image, 2))
    iteration += 1
    

Startbild:
[[2.75 2.75]
 [2.75 2.75]]
Iteration: 1
[[2.91 1.32]
 [4.86 1.82]]
Iteration: 2
[[2.81 1.2 ]
 [5.05 1.75]]
Iteration: 3
[[2.81 1.18]
 [5.07 1.76]]
